<a href="https://colab.research.google.com/github/jsaroni/QHack2023_Feynmanprodigies/blob/main/BeH2_gs_vqe_frozen_orb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install qiskit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 46.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 54.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.3/241.3 KB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.0/107.0 KB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 KB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 KB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.5/37.5 MB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 KB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 59.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 K

In [9]:
!pip install qiskit_nature

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 34.6 MB/s eta 0:00:00


In [17]:
!pip install pyscf

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [18]:
!pip install git+https://github.com/pyscf/pyscf

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/pyscf/pyscf to /tmp/pip-req-build-hb_lugad
  Running command git clone --filter=blob:none --quiet https://github.com/pyscf/pyscf /tmp/pip-req-build-hb_lugad
  Resolved https://github.com/pyscf/pyscf to commit 8b3fef8cf18f10d430261d4a8bea21fadf19bb1f
  Running command git submodule update --init --recursive -q
  Preparing metadata (setup.py) ... done


In [1]:
# pylint: disable=missing-function-docstring
import numpy as np
np.random.seed(999999)
p0 = np.random.random()
target_distr = {0: p0, 1: 1-p0}

In [2]:
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
def get_var_form(params):
    qr = QuantumRegister(1, name="q")
    cr = ClassicalRegister(1, name='c')
    qc = QuantumCircuit(qr, cr)
    qc.u(params[0], params[1], params[2], qr[0])
    qc.measure(qr, cr[0])
    return qc

In [3]:
from qiskit import Aer
backend = Aer.get_backend("aer_simulator")

def counts_to_distr(counts):
    """Convert Qiskit result counts to dict with integers as
    keys, and pseudo-probabilities as values."""
    n_shots = sum(counts.values())
    return {int(k, 2): v/n_shots for k, v in counts.items()}

def objective_function(params):
    """Compares the output distribution of our circuit with
    parameters `params` to the target distribution."""
    # Create circuit instance with paramters and simulate it
    qc = get_var_form(params)
    result = backend.run(qc).result()
    # Get the counts for each measured state, and convert
    # those counts into a probability dict
    output_distr = counts_to_distr(result.get_counts())
    # Calculate the cost as the distance between the output
    # distribution and the target distribution
    cost = sum(
        abs(target_distr.get(i, 0) - output_distr.get(i, 0))
        for i in range(2**qc.num_qubits)
    )
    return cost

In [4]:
from qiskit.algorithms.optimizers import COBYLA
optimizer = COBYLA(maxiter=500, tol=0.0001)

# Create the initial parameters (noting that our
# single qubit variational form has 3 parameters)
params = np.random.rand(3)
result = optimizer.minimize(
    fun=objective_function,
    x0=params)

# Obtain the output distribution using the final parameters
qc = get_var_form(result.x)
counts = backend.run(qc, shots=10000).result().get_counts()
output_distr = counts_to_distr(counts)

print("Parameters Found:", result.x)
print("Target Distribution:", target_distr)
print("Obtained Distribution:", output_distr)
print("Cost:", objective_function(result.x))

Parameters Found: [ 1.9420242   0.2252642  -0.27603228]
Target Distribution: {0: 0.308979188922057, 1: 0.691020811077943}
Obtained Distribution: {1: 0.6773, 0: 0.3227}
Cost: 0.030479122155885996


In [5]:
from qiskit.circuit.library import EfficientSU2
entanglements = ["linear", "full"]
for entanglement in entanglements:
    form = EfficientSU2(num_qubits=4, entanglement=entanglement)
    print(f"{entanglement} entanglement:")
    # We initialize all parameters to 0 for this demonstration
    display(form.decompose().draw(fold=-1))

linear entanglement:


┌──────────┐┌──────────┐     ┌──────────┐┌───────────┐                          ┌───────────┐┌───────────┐                          ┌───────────┐┌───────────┐                          
q_0: ┤ Ry(θ[0]) ├┤ Rz(θ[4]) ├──■──┤ Ry(θ[8]) ├┤ Rz(θ[12]) ├───────────────────■──────┤ Ry(θ[16]) ├┤ Rz(θ[20]) ├───────────────────■──────┤ Ry(θ[24]) ├┤ Rz(θ[28]) ├──────────────────────────
     ├──────────┤├──────────┤┌─┴─┐└──────────┘└┬──────────┤┌───────────┐    ┌─┴─┐    └───────────┘├───────────┤┌───────────┐    ┌─┴─┐    └───────────┘├───────────┤┌───────────┐             
q_1: ┤ Ry(θ[1]) ├┤ Rz(θ[5]) ├┤ X ├─────■───────┤ Ry(θ[9]) ├┤ Rz(θ[13]) ├────┤ X ├──────────■──────┤ Ry(θ[17]) ├┤ Rz(θ[21]) ├────┤ X ├──────────■──────┤ Ry(θ[25]) ├┤ Rz(θ[29]) ├─────────────
     ├──────────┤├──────────┤└───┘   ┌─┴─┐     └──────────┘├───────────┤┌───┴───┴───┐    ┌─┴─┐    └───────────┘├───────────┤┌───┴───┴───┐    ┌─┴─┐    └───────────┘├───────────┤┌───────────┐
q_2: ┤ Ry(θ[2]) ├┤ Rz(θ[6]) ├────────┤ X ├──────────■──────┤ Ry(θ[10]) ├┤ Rz(θ[14]) ├────┤ X ├──────────■──────┤ Ry(θ[18]) ├┤ Rz(θ[22]) ├────┤ X ├──────────■──────┤ Ry(θ[26]) ├┤ Rz(θ[30]) ├
     ├──────────┤├──────────┤        └───┘        ┌─┴─┐    ├───────────┤├───────────┤    └───┘        ┌─┴─┐    ├───────────┤├───────────┤    └───┘        ┌─┴─┐    ├───────────┤├───────────┤
q_3: ┤ Ry(θ[3]) ├┤ Rz(θ[7]) ├─────────────────────┤ X ├────┤ Ry(θ[11]) ├┤ Rz(θ[15]) ├─────────────────┤ X ├────┤ Ry(θ[19]) ├┤ Rz(θ[23]) ├─────────────────┤ X ├────┤ Ry(θ[27]) ├┤ Rz(θ[31]) ├
     └──────────┘└──────────┘                     └───┘    └───────────┘└───────────┘                 └───┘    └───────────┘└───────────┘                 └───┘    └───────────┘└───────────┘

full entanglement:


┌──────────┐┌──────────┐                    ┌──────────┐┌───────────┐                                         ┌───────────┐┌───────────┐                                         ┌───────────┐┌───────────┐                          
q_0: ┤ Ry(θ[0]) ├┤ Rz(θ[4]) ├──■────■─────────■──┤ Ry(θ[8]) ├┤ Rz(θ[12]) ├───────────────────■────────■─────────■──┤ Ry(θ[16]) ├┤ Rz(θ[20]) ├───────────────────■────────■─────────■──┤ Ry(θ[24]) ├┤ Rz(θ[28]) ├──────────────────────────
     ├──────────┤├──────────┤┌─┴─┐  │         │  └──────────┘└┬──────────┤┌───────────┐    ┌─┴─┐      │         │  └───────────┘├───────────┤┌───────────┐    ┌─┴─┐      │         │  └───────────┘├───────────┤┌───────────┐             
q_1: ┤ Ry(θ[1]) ├┤ Rz(θ[5]) ├┤ X ├──┼────■────┼───────■───────┤ Ry(θ[9]) ├┤ Rz(θ[13]) ├────┤ X ├──────┼────■────┼────────■──────┤ Ry(θ[17]) ├┤ Rz(θ[21]) ├────┤ X ├──────┼────■────┼────────■──────┤ Ry(θ[25]) ├┤ Rz(θ[29]) ├─────────────
     ├──────────┤├──────────┤└───┘┌─┴─┐┌─┴─┐  │       │       └──────────┘├───────────┤┌───┴───┴───┐┌─┴─┐┌─┴─┐  │        │      └───────────┘├───────────┤┌───┴───┴───┐┌─┴─┐┌─┴─┐  │        │      └───────────┘├───────────┤┌───────────┐
q_2: ┤ Ry(θ[2]) ├┤ Rz(θ[6]) ├─────┤ X ├┤ X ├──┼───────┼────────────■──────┤ Ry(θ[10]) ├┤ Rz(θ[14]) ├┤ X ├┤ X ├──┼────────┼────────────■──────┤ Ry(θ[18]) ├┤ Rz(θ[22]) ├┤ X ├┤ X ├──┼────────┼────────────■──────┤ Ry(θ[26]) ├┤ Rz(θ[30]) ├
     ├──────────┤├──────────┤     └───┘└───┘┌─┴─┐   ┌─┴─┐        ┌─┴─┐    ├───────────┤├───────────┤└───┘└───┘┌─┴─┐    ┌─┴─┐        ┌─┴─┐    ├───────────┤├───────────┤└───┘└───┘┌─┴─┐    ┌─┴─┐        ┌─┴─┐    ├───────────┤├───────────┤
q_3: ┤ Ry(θ[3]) ├┤ Rz(θ[7]) ├───────────────┤ X ├───┤ X ├────────┤ X ├────┤ Ry(θ[11]) ├┤ Rz(θ[15]) ├──────────┤ X ├────┤ X ├────────┤ X ├────┤ Ry(θ[19]) ├┤ Rz(θ[23]) ├──────────┤ X ├────┤ X ├────────┤ X ├────┤ Ry(θ[27]) ├┤ Rz(θ[31]) ├
     └──────────┘└──────────┘               └───┘   └───┘        └───┘    └───────────┘└───────────┘          └───┘    └───┘        └───┘    └───────────┘└───────────┘          └───┘    └───┘        └───┘    └───────────┘└───────────┘

In [6]:
# pylint: disable=line-too-long
from qiskit.algorithms import VQE
from qiskit_nature.algorithms import (GroundStateEigensolver,
                                      NumPyMinimumEigensolverFactory)
from qiskit_nature.drivers import Molecule
from qiskit_nature.drivers.second_quantization import (
    ElectronicStructureMoleculeDriver, ElectronicStructureDriverType)
from qiskit_nature.transformers.second_quantization.electronic import FreezeCoreTransformer
from qiskit_nature.problems.second_quantization import ElectronicStructureProblem
from qiskit_nature.converters.second_quantization import QubitConverter
from qiskit_nature.mappers.second_quantization import ParityMapper
# pylint: enable=line-too-long

import matplotlib.pyplot as plt
import numpy as np
from qiskit_nature.circuit.library import UCCSD, HartreeFock
from qiskit.circuit.library import EfficientSU2
from qiskit.algorithms.optimizers import COBYLA, SPSA, SLSQP
from qiskit.opflow import TwoQubitReduction
from qiskit import BasicAer, Aer
from qiskit.utils import QuantumInstance
from qiskit.utils.mitigation import CompleteMeasFitter
from qiskit.providers.aer.noise import NoiseModel

<ipython-input-6-2969ff79605f>:3: NatureDeprecationWarning: The qiskit_nature.algorithms.excited_state_solvers package is deprecated as of version 0.5.0 and will be removed no sooner than 3 months after the release. Instead use the qiskit_nature.second_q.algorithms.excited_state_solvers package.
  from qiskit_nature.algorithms import (GroundStateEigensolver,
<ipython-input-6-2969ff79605f>:3: NatureDeprecationWarning: The qiskit_nature.algorithms.pes_sampler package is deprecated as of version 0.5.0 and will be removed no sooner than 3 months after the release . Please refer to 'https://github.com/Qiskit/qiskit-nature/issues/750' for additional information regarding 'extrapolators'.
  from qiskit_nature.algorithms import (GroundStateEigensolver,


In [7]:
from qiskit_nature.settings import settings
settings.dict_aux_operators = False

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: ListAuxOpsDeprecationWarning: List-based `aux_operators` are deprecated as of version 0.3.0 and support for them will be removed no sooner than 3 months after the release. Instead, use dict-based `aux_operators`. You can switch to the dict-based interface immediately, by setting `qiskit_nature.settings.dict_aux_operators` to `True`.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [8]:
def get_qubit_op(dist):
    # Define Molecule
    molecule = Molecule(
        # Coordinates in Angstrom
        geometry=[
            ["Be", [0.0, 0.0, 0.0]],
            ["H", [-dist, 0.0, 0.0]],
            ["H", [dist, 0.0, 0.0]]
        ],
        multiplicity=1,  # = 2*spin + 1
        charge=0,
    )

    driver = ElectronicStructureMoleculeDriver(
        molecule=molecule,
        basis="sto3g",
        driver_type=ElectronicStructureDriverType.PYSCF)
    

    driver = ElectronicStructureMoleculeDriver(
         molecule=molecule,
         basis="sto3g",
        driver_type=ElectronicStructureDriverType.PYSCF)

    # Define Problem, Use freeze core approximation, remove orbitals.
    from qiskit_nature.problems.second_quantization import ElectronicStructureProblem
    from qiskit_nature.transformers.second_quantization.electronic import FreezeCoreTransformer
    problem = ElectronicStructureProblem(
        driver,
        [FreezeCoreTransformer(freeze_core=True)])#, remove_orbitals=[-3,-2,-1,0])])



    # Get properties
    properties = driver.run()
    num_particles = (properties
                        .get_property("ParticleNumber")
                        .num_particles)
    num_spin_orbitals = int(properties
                            .get_property("ParticleNumber")
                            .num_spin_orbitals)

    # Define Problem, Use freeze core approximation, remove orbitals.
    problem = ElectronicStructureProblem(
        driver,
        [FreezeCoreTransformer(freeze_core=True,
                               remove_orbitals=[-3,-2])])

    second_q_ops = problem.second_q_ops()  # Get 2nd Quant OP
    num_spin_orbitals = problem.num_spin_orbitals
    num_particles = problem.num_particles

    mapper = ParityMapper()  # Set Mapper
    hamiltonian = second_q_ops[0]  # Set Hamiltonian
    # Do two qubit reduction
    converter = QubitConverter(mapper,two_qubit_reduction=True)
    reducer = TwoQubitReduction(num_particles)
    qubit_op = converter.convert(hamiltonian)
    qubit_op = reducer.convert(qubit_op)

    return qubit_op, num_particles, num_spin_orbitals, problem, converter

In [9]:
def exact_solver(problem, converter):
    solver = NumPyMinimumEigensolverFactory()
    calc = GroundStateEigensolver(converter, solver)
    result = calc.solve(problem)
    return result

backend = BasicAer.get_backend("statevector_simulator")
distances = np.arange(0.5, 4.0, 0.2)
exact_energies = []
vqe_energies = []
optimizer = SLSQP(maxiter=5)

# pylint: disable=undefined-loop-variable


dist = 1.3
(qubit_op, num_particles, num_spin_orbitals,
                            problem, converter) = get_qubit_op(dist)
result = exact_solver(problem,converter)
exact_energies.append(result.total_energies[0].real)
init_state = HartreeFock(num_spin_orbitals, num_particles, converter)
var_form = UCCSD(converter,
                    num_particles,
                    num_spin_orbitals,
                    initial_state=init_state)
vqe = VQE(var_form, optimizer, quantum_instance=backend)
vqe_calc = vqe.compute_minimum_eigenvalue(qubit_op)
vqe_result = problem.interpret(vqe_calc).total_energies[0].real
vqe_energies.append(vqe_result)
print(f"Interatomic Distance: {np.round(dist, 2)}",
        f"VQE Result: {vqe_result:.5f}",
        f"Exact Energy: {exact_energies[-1]:.5f}")

print("All energies have been calculated")

<ipython-input-9-1e71ef5baea6>:18: NatureDeprecationWarning: The qiskit_nature.drivers.Molecule class is deprecated as of version 0.5.0 and will be removed no sooner than 3 months after the release. Instead use the qiskit_nature.second_q.drivers.Molecule class.
  problem, converter) = get_qubit_op(dist)
<ipython-input-9-1e71ef5baea6>:18: NatureDeprecationWarning: The qiskit_nature.problems.second_quantization.electronic.ElectronicStructureProblem class is deprecated as of version 0.5.0 and will be removed no sooner than 3 months after the release. Instead use the qiskit_nature.second_q.problems.ElectronicStructureProblem class.
  problem, converter) = get_qubit_op(dist)
<ipython-input-9-1e71ef5baea6>:3: NatureDeprecationWarning: The qiskit_nature.algorithms.ground_state_solvers.GroundStateEigensolver class is deprecated as of version 0.5.0 and will be removed no sooner than 3 months after the release. Instead use the qiskit_nature.second_q.algorithms.ground_state_solvers.GroundStateEig

Interatomic Distance: 1.3 VQE Result: -15.59436 Exact Energy: -15.59471
All energies have been calculated
